In [1]:
!nvidia-smi

Wed Apr  3 08:43:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.5 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.5/821.5 kB 4.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 6.4 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 23.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 24.5 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 5.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 1.6 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-z8vj41th/overlay/local/bin
    changing mode of /tmp/pip-build-env-z8vj41th/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-z8vj41th/overlay/loc

In [3]:
model_name_or_path = "TheBloke/llama-2-13B-chat-limarp-v2-merged-GGML"
model_basename = "llama-2-13b-chat-limarp-v2-merged.ggmlv3.Q8_0.bin" # the model is in bin forma

In [4]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [5]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

(…)3b-chat-limarp-v2-merged.ggmlv3.Q8_0.bin:   0%|          | 0.00/13.8G [00:00<?, ?B/s]

In [6]:
model_path

'/root/.cache/huggingface/hub/models--TheBloke--llama-2-13B-chat-limarp-v2-merged-GGML/snapshots/3bac80e90010216f02d683c5be69cbb4af7249be/llama-2-13b-chat-limarp-v2-merged.ggmlv3.Q8_0.bin'

In [7]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [8]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

In [9]:
prompt = "Write a linear regression code"
prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: {prompt}

ASSISTANT:
'''

In [10]:
response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

In [11]:
print(response)

{'id': 'cmpl-d862c9b6-529f-4fa9-b1c3-a8bae99aca4e', 'object': 'text_completion', 'created': 1712134474, 'model': '/root/.cache/huggingface/hub/models--TheBloke--llama-2-13B-chat-limarp-v2-merged-GGML/snapshots/3bac80e90010216f02d683c5be69cbb4af7249be/llama-2-13b-chat-limarp-v2-merged.ggmlv3.Q8_0.bin', 'choices': [{'text': "SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.\n\nUSER: Write a linear regression code\n\nASSISTANT:\nI'd be happy to help! However, I need some more information about the project you have in mind. Could you please provide me with the following details:\n\n1. What is the dependent variable (Y) that you want to predict?\n2. What are the independent variables (X) that you think will most strongly influence Y?\n3. Do you have any missing values or outliers in your data that need to be handled?\n4. Is there a specific type of linear regression model you would like to use (e.g., simple linear regression, multiple linear regression)

In [12]:
print(response["choices"][0]["text"])

SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: Write a linear regression code

ASSISTANT:
I'd be happy to help! However, I need some more information about the project you have in mind. Could you please provide me with the following details:

1. What is the dependent variable (Y) that you want to predict?
2. What are the independent variables (X) that you think will most strongly influence Y?
3. Do you have any missing values or outliers in your data that need to be handled?
4. Is there a specific type of linear regression model you would like to use (e.g., simple linear regression, multiple linear regression)?
5. What programming language and software are you using for this project?

Once I have more information about your project, I can provide you with the code for the linear regression analysis.
